# LIBRARIES

In [ ]:
import torch
import json
import os
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import scipy
import collections
from PIL import Image
import pickle
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.mobilenetv2 import Conv2dNormActivation, InvertedResidual
import torch.optim as optim
import torch.nn.functional as F
import itertools
from torch.optim.lr_scheduler import CosineAnnealingLR
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score


# CUDA

In [ ]:
# Train on GPU if available
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device = torch.device("cuda:0" if train_on_gpu else "cpu")

print(device)

# IMPORT THE DATASET

In [ ]:
'''

This part of the code serves the purpose of dowloading
the dataset in a proper way for kaggle. A file json is created
to download the dataset from kaggle.

'''

# Create file kaggle.json to download from Kaggle
kaggle_json = {
    "username": "enricomarta",
    "key": "84d386ad7a7c02ed584cae00085f289b"
}

# Path of the kaggle.json
kaggle_json_path = '/kaggle/working/kaggle.json'

# Write on kaggle.json
with open(kaggle_json_path, 'w') as file:
    json.dump(kaggle_json, file)

# Kaggle's enviornment
os.environ['KAGGLE_CONFIG_DIR'] = '/kaggle/working'

# Correctness of the information
!chmod 600 /kaggle/working/kaggle.json

print("Downloading the dataset from Kaggle...")
!kaggle competitions download -c ifood-2019-fgvc6 -p /kaggle/working --force

In [ ]:
# unzip files
!unzip -o ifood-2019-fgvc6.zip -d /kaggle/working > /dev/null
!unzip -o train_set.zip -d /kaggle/working > /dev/null

!ls

In [ ]:
# Remove unnecesary files for the project
!rm class_list.txt ifood-2019-fgvc6.zip sample_submission.csv ifood2019_sample_submission.csv test_set.zip train_set.zip val_labels.csv val_set.zip

# PREPROCESSING

In [ ]:
'''
Here training and validation set are created and saved.
Then number of images contained in both are providded.
'''

# path of the csv file
file_path = '/kaggle/working/train_labels.csv'
data = pd.read_csv(file_path)

# Split the data into training and validation sets (30% for validation)
train_data, validation_data = train_test_split(data, test_size=0.3, stratify=data['label'], random_state=42)

# Save the validation set
validation_data.to_csv('validation_set.csv', index=False)

# Save the training set
train_data.to_csv('training_set.csv', index=False)

num_train_rows = train_data.shape[0]
num_validation_rows = validation_data.shape[0]
print(f"Number of rows in the training set: {num_train_rows}")
print(f"Number of rows in the validation set: {num_validation_rows}")

In [ ]:
'''
Here the 'final dataset' is created, augmenting
under-represented classes via duplication, and then saved.
'''
# Path of the CSV file
file_path = '/kaggle/working/training_set.csv'

# Upload the CSV file
data = pd.read_csv(file_path)

# Number of rows
num_rows = data.shape[0]
print(f"Number of rows: {num_rows}")

# Number of labels
num_labels = data['label'].nunique()
print(f"Number of labels: {num_labels}")

# Number of observations for each label
images_per_label = data['label'].value_counts()
print("Number of images per label (ascending order):")
print(images_per_label.sort_values())

# Find classes with fewer than 300 observations
classes_to_duplicate = images_per_label[images_per_label < 300].index

# DataFrame for the duplicate images
duplicated_data = []

# For each label that doesn't have enough images
for label in classes_to_duplicate:
    rows_to_duplicate = data[data['label'] == label]
    # Calculate how many more images are needed to reach 300
    count_needed = 300 - len(rows_to_duplicate)
    # Generates a sample of duplicate rows from the current class
    duplicated_rows = rows_to_duplicate.sample(n=count_needed, replace=True, random_state=1)
    # Concatenates the original rows together with the duplicated rows
    duplicated_data.append(pd.concat([rows_to_duplicate, duplicated_rows], ignore_index=True))

# Merge all positions contained in the list
final_duplicated_data = pd.concat(duplicated_data, ignore_index=True)

# Well represented data (already have 300 or more observations)
well_represented_data = data[data['label'].value_counts()[data['label']].values >= 300]

# Merge to obtain the final new dataframe
final_data = pd.concat([well_represented_data, final_duplicated_data], ignore_index=True)

# Create the new file CSV
final_file_path = '/kaggle/working/modified_train_labels.csv'
final_data.to_csv(final_file_path, index=False)

# New Training Dataset
data_modified = pd.read_csv(final_file_path)
num_rows_modified = len(data_modified)
print(f"Number of images in modified file: {num_rows_modified}")
images_per_label_mod = data_modified['label'].value_counts()
print("Number of images per label (ascending order):")
print(images_per_label_mod.sort_values())

In [ ]:
'''
Here the class 'FoodDataset is created.
It provides an appropriate way to manage images.
'''
class FoodDataset(Dataset):

    def __init__(self, root_dir, transform, split):
        """
        Args:
            root_dir (string): Directory containing the images.
            transform (callable, optional): Transformation to be applied to the images.
            split (string): Split type ("train" or "val").
        """
        self.split = split
        self.root_dir = root_dir
        self.data = self.get_data_train()
        self.data_v = self.get_data_val()

        if split == "train":
            self.data = self.data
        elif split == "val":
            self.data = self.data_v
        self.transform = transform

    def get_data_train(self):
        df = pd.read_csv('/kaggle/working/training_set.csv')
        # Construct full image paths using vectorized operations
        df['img_name'] = self.root_dir + "/" + df['img_name']
        # Filter for images with existing paths using vectorized boolean indexing
        df = df[df['img_name'].apply(os.path.exists)]
        return df.to_numpy()  # Return preprocessed data as a NumPy array

    def get_data_val(self):
        df = pd.read_csv('/kaggle/working/validation_set.csv')
        # Construct full image paths using vectorized operations
        df['img_name'] = self.root_dir + "/" + df['img_name']
        # Filter for images with existing paths using vectorized boolean indexing
        df = df[df['img_name'].apply(os.path.exists)]
        return df.to_numpy()  # Return preprocessed data as a NumPy array

    def __len__(self):
        """
        Returns the length of the dataset (number of samples).

        This method overrides the default behavior of `len` for the dataset object.
        It simply returns the length of the internal `data` list, which represents
        the preprocessed data after loading and filtering.
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieves a sample at a given index.

        This method overrides the default behavior of indexing for the dataset object.
        It takes an index `idx` and performs the following:
            1. Accesses the image name and num_class at the specified index from `self.data`.
            2. Opens the image using `Image.open` with the full path constructed by
               combining `self.root_dir` and `img_name`.
            3. Applies the defined transformation (`self.transform`) to the image.
            4. Creates a dictionary `sample` containing the preprocessed image (`image`)
               and the num_class as a PyTorch tensor (`torch.tensor(num_class).float()`).
            5. Returns the constructed `sample` dictionary.
        """
        img_name, num_class = self.data[idx]
        image = Image.open(os.path.join(self.root_dir, img_name))
        image = self.transform(image)

        sample = {'img_name': image, 'labels': torch.tensor(num_class).float()}
        return sample


In [ ]:
'''
Functions for data transformation are provided,
distinguishing between trainin and validation sets.
Creation of Dataloaders.
'''
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to 256x256
    #transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip images horizontally with a probability of 50%
    #transforms.RandomAffine(degrees=0, shear=0.2),  # Apply random shear transformations
    #transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0), ratio=(1.0, 1.0)),  # Apply random zoom transformations
    transforms.ToTensor(),  # Convert PIL images to PyTorch tensors
    transforms.Normalize(  # Normalize pixel values based on ImageNet statistics
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
transform_val = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to 256x256 (consistent with training)
    transforms.ToTensor(),  # Convert PIL images to PyTorch tensors
    transforms.Normalize(  # Normalize pixel values using the same statistics
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


# Set batch size
bs = 30

# Create datasets for training and validation
trainset = FoodDataset("/kaggle/working/train_set", transform_train, split="train")
valset = FoodDataset("/kaggle/working/train_set", transform_val, split="val")

# Create data loaders for efficient batch training and evaluation
trainloader = torch.utils.data.DataLoader(trainset, batch_size=bs, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=1, shuffle=False)

# Print dataset and dataloader lengths (number of samples and batches)
print(f"Number of training samples: {len(trainloader) * bs}")
print(f"Number of validation samples: {len(valloader)}")

# CNN

In [ ]:
'''
Here the architecture of the CNN is provided.
Functions for SSL pretext task are present.
'''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Let's create a modified version of MobileNetV2
        self.features = nn.Sequential(
            Conv2dNormActivation(3, 32, kernel_size=3, stride=2, padding=1),
            InvertedResidual(32, 16, stride=1, expand_ratio=1),
            InvertedResidual(16, 24, stride=2, expand_ratio=6),
            InvertedResidual(24, 24, stride=1, expand_ratio=6),
            InvertedResidual(24, 32, stride=2, expand_ratio=6),
            InvertedResidual(32, 32, stride=1, expand_ratio=6),
            InvertedResidual(32, 32, stride=1, expand_ratio=6),
            InvertedResidual(32, 64, stride=2, expand_ratio=6),
            InvertedResidual(64, 64, stride=1, expand_ratio=6),
            InvertedResidual(64, 64, stride=1, expand_ratio=6),
            InvertedResidual(64, 64, stride=1, expand_ratio=6),
            InvertedResidual(64, 96, stride=1, expand_ratio=6),
            InvertedResidual(96, 96, stride=2, expand_ratio=6),
            InvertedResidual(96, 96, stride=2, expand_ratio=6),
            Conv2dNormActivation(96, 800, kernel_size=1, stride=1, padding=0)
        )

        # Definition of classifier
        self.classifier_puzzle = self._create_classifier(4)

    def _create_classifier(self, num_classes):
        return nn.Sequential(
            nn.Linear(800, 445),
            nn.GELU(),
            nn.Linear(445, 32),
            nn.GELU(),
            nn.Linear(32, num_classes)
        )
    def forward_shared(self, x):
        x = self.features(x)
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1)).flatten(1)
        return x

    def forward_puzzle(self, x):
        x = self.forward_shared(x)
        x = self.classifier_puzzle(x)
        return x


    def _preprocess(self, images):
        """
        Preprocess the input images to generate rotated images and create corresponding targets.

        Parameters:
            images (Tensor): Batch of input images.

        Returns:
            images_batch (Tensor): Batch of rotated images.
            targets (Tensor): Corresponding targets for the original image rotations.
        """
        batch_size = images.shape[0]

        # Generate rotated images
        images_90 = torch.flip(images.transpose(2, 3), (2,))
        images_180 = torch.flip(images, (2, 3))
        images_270 = torch.flip(images, (2,)).transpose(2, 3)

        # Concatenate original and rotated images
        images_batch = torch.cat((images, images_90, images_180, images_270), dim=0)

        # Create targets for rotations
        targets = torch.arange(4).long().repeat(batch_size)
        targets = targets.view(batch_size, 4).transpose(0, 1)
        targets = targets.contiguous().view(-1)

        return images_batch, targets

net = Net()
net.to(device)


# TRAINING and EVALUATION

In [ ]:
'''
Define training parameters (epochs, loss function, optimizer, and scheduler)
'''
epochs = 12  # Number of training epochs

criterion = nn.CrossEntropyLoss()
optimizer_shared = optim.Adam(net.features.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001
scheduler_shared = CosineAnnealingLR(optimizer_shared,
                              T_max=len(trainloader) * epochs,  # Maximum number of iterations for scheduler
                              eta_min=1e-5)  # Minimum learning rate for scheduler
optimizer_puzzle = optim.Adam(net.classifier_puzzle.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001
scheduler_puzzle = CosineAnnealingLR(optimizer_puzzle,
                              T_max=len(trainloader) * epochs,  # Maximum number of iterations for scheduler
                              eta_min=1e-5)  # Minimum learning rate for scheduler


In [ ]:
'''
Training and Validation section.
'''

all_preds = []
all_labels = []

# Training loop
for epoch in range(epochs):  # Loop over the dataset for multiple epochs
    running_loss = []  # List to store training loss for each batch
    net.train()  # Set the model to training mode
    for i, data in enumerate(trainloader):
        # Get inputs and labels from the data loader
        inputs, labels = data["img_name"], data["labels"]
        inputs = inputs.to(device)
        labels = labels.to(device).long()
        # Zero the parameter gradients before each backward pass
        optimizer_shared.zero_grad()
        optimizer_puzzle.zero_grad()
        # Forward pass
        shuffled_pieces, targets = net._preprocess(inputs)
        shuffled_pieces, targets = shuffled_pieces.to(device), targets.to(device)

        outputs = net.forward_puzzle(shuffled_pieces)
        outputs = outputs.to(device)
        # Calculate Loss
        loss = criterion(outputs.squeeze(), targets)
        # Backward pass and parameter update
        loss.backward()
        optimizer_shared.step()
        optimizer_puzzle.step()
        scheduler_shared.step()
        scheduler_puzzle.step()
        # Print statistics (every 10% of the training data)
        running_loss.append(loss.item())

        if (i + 1) % (len(trainloader) // 10) == 0:  # Every 10% of the epoch
            print('%d, [%d, %d] loss puzzle: %.4f\tlr: %.6f' %
                  (epoch + 1, i + 1, len(trainloader), np.mean(running_loss), optimizer_puzzle.param_groups[-1]['lr']))
            running_loss = []
    # Validation loop
    running_loss = []  # List to store validation loss for each batch

    net.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    for i, data in enumerate(valloader):
        # Get inputs and labels from the data loader
        inputs, labels = data["img_name"], data["labels"]
        inputs = inputs.to(device)
        labels = labels.to(device).long()

        shuffled_pieces, targets = net._preprocess(inputs)
        shuffled_pieces, targets = shuffled_pieces.to(device), targets.to(device)
        # Forward pass with gradient suppression
        with torch.no_grad():
            outputs = net.forward_puzzle(shuffled_pieces)  # Get model predictions without calculating gradients
            outputs = outputs.to(device)

            # Finds the class with the highest probability for each image in the batch.
            _, predicted = torch.max(outputs.cpu(), 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())
            # Update total number of test images
            total += targets.size(0)  # label.size(0) gives the batch size
            # Count correct predictions
            correct += (predicted == targets.cpu()).sum().item()  # Count true positives


        loss = criterion(outputs, targets)  # Calculate loss
        running_loss.append(loss.item())

    val_loss = np.mean(running_loss)

    print('Validation loss puzzle: %.6f' % (val_loss))
    # Calculate and print accuracy
    accuracy = 100 * correct / total
    print(f'Validation accuracy puzzle: {accuracy:.2f} %')



print('Finished Training')
torch.save(net.state_dict(), "net_last.pth")  # Save the last model state

# FEATURES EXTRACTION

In [ ]:
'''
Extract features from the model
'''

def extract_features(model, dataloader, device):
    model.eval() # Evaluation mode
    # List to save features and labels
    features = []
    labels = []
    # Gradient suppression
    with torch.no_grad():
        for data in dataloader:
            inputs, lbls = data["img_name"], data["labels"]
            inputs = inputs.to(device)
            lbls = lbls.to(device).long()
            # Forward pass
            feats = model.forward_shared(inputs)
            features.append(feats.cpu().numpy())
            labels.append(lbls.cpu().numpy())
    # Creation of the final lists
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)

    return features, labels

In [ ]:
'''
Upload the model and the features
'''

model = Net()
model.load_state_dict(torch.load('net_last.pth'))
model.to(device)

features, labels = extract_features(model, valloader, device)

np.save('puzzle_features.npy', features)
np.save('puzzle_labels.npy', labels)

print('Done')

# SVM

In [ ]:
'''
Training of the SVM with extracted features.
Evaluation.
'''
features = np.load('puzzle_features.npy')
labels = np.load('puzzle_labels.npy')

#(80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Initialize SVM
classifier = svm.SVC(kernel='linear')

# Training
classifier.fit(X_train, y_train)

# Prediction
y_pred = classifier.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy * 100:.2f}%')